<a href="https://colab.research.google.com/github/suresh-venkate/NLP_LLM/blob/main/Courses/HF_Transformers_Course/Sec_2P2_Behind_The_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace - Behind the pipeline API (PyTorch)

There are three steps in a transformer pipeline:



1.   Preprocessing with tokenizers
2.   Passing the tokenized inputs through the model
3.   Postprocessing

# Installations

In [10]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]

# Pipeline API usage

In [11]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

# AutoTokenizer



1. This is the first step of the pipeline.
2. Transformer models can’t process raw text directly, so the first step of our pipeline is to convert the text inputs into numbers that the model can make sense of. To do this we use a tokenizer, which will be responsible for:
  

  *   Splitting the input into words, subwords, or symbols (like punctuation) that are called tokens.
  *   Mapping each token to an integer
  *   Adding additional inputs that may be useful to the model

All this preprocessing needs to be done in exactly the same way as when the model was pretrained, so we first need to download that information from the Model Hub. To do this, we use the AutoTokenizer class and its from_pretrained() method. Using the checkpoint name of our model, it will automatically fetch the data associated with the model’s tokenizer and cache it.

The default checkpoint of the sentiment-analysis pipeline is distilbert-base-uncased-finetuned-sst-2-english. Once we have the tokenizer, we can directly pass our sentences to it and we’ll get back a dictionary that’s ready to feed to our model! The only thing left to do is to convert the list of input IDs to tensors.


In [12]:
%%capture

from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [13]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


# Process tokens through model

The class 'AutoModel' used below contains only the base transformer model without the final processing head. Its output generally has three dimensions:




*   Batch size: The number of sequences processed at a time (2 in our example).
*   Sequence length: The length of the numerical representation of the sequence (16 in our example).
*   Hidden size: The vector dimension of each model input.

In [14]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [15]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


The class 'AutoModelForSequenceClassification' used below adds a classification head to the above model output. This will output logits.

In [16]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [17]:
print(outputs.logits.shape) # Two sentences and two labels. So, output is of size 2 x 2.

torch.Size([2, 2])


In [18]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


# Postprocess Model Outputs

In [19]:
# Convert logits to probabilities by applying softmax function
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [20]:
# To get the labels corresponding to each position, we can inspect the id2label attribute of the model config
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}